In [1]:
from glob import glob
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import cv2
import pickle
import warnings
warnings.filterwarnings('ignore')

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import time
import albumentations as albu
from albumentations.pytorch import ToTensorV2

import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour, plot_optimization_history
from optuna.visualization import plot_parallel_coordinate, plot_slice, plot_param_importances
from lightgbm import LGBMClassifier

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
def load_dict(model):
    with open('../pickle/pred_dict_'+model+'.pickle', 'rb') as fw:
        pred_dict_new_local = pickle.load(fw)
    with open('../pickle/pred_test_dict_'+model+'.pickle', 'rb') as fw:
        pred_test_dict_new_local = pickle.load(fw)
    return pred_dict_new_local, pred_test_dict_new_local

In [3]:
all_img_path = np.array(sorted(glob('../data/train/*.png')))
test_img_path = np.array(sorted(glob('../data/test/*.png')))

In [5]:
train_y = pd.read_csv("../data/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

all_label = np.array([label_unique[k] for k in train_labels])

In [6]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [7]:
pred_dict_lgb, pred_test_dict_lgb = load_dict('lgb')

In [14]:
pred_dict_efficientb0, pred_test_dict_efficientb0 = load_dict('efficientb0')